In [1]:
import numpy as np
import scipy.io as sp
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import math 



In [2]:
def model(A, THETA):
    """
    Multivariate Linear Regression Model, Yh = A*THETA
    The matrix A is sometimes called the design matrix.
    """
    return A.dot(THETA)


In [3]:

# Design Matrix
def designMatrix(Tau,X):
    q,_ = X.shape
    for p in range(q):
        M = powerVector(Tau,X[p,:])
        if p == 0:
            A = M
            continue
        A = np.vstack((A,M))
    return A

# Power Vector M
def powerVector(Tau, V):
    if V.size == 0 or Tau == 0:
        return np.array([[1.0]])  # Asegurar 2D
    Z = V[:-1]
    W = V[-1]
    terms = []
    for k in range(Tau + 1):
        sub_terms = powerVector(Tau - k, Z)
        for term in sub_terms.flatten():  # Manejar sub_terms 2D
            terms.append(term * (W ** k))
    return np.array([terms])  # Devolver como 2D (1, n_terms)


In [4]:
#Polynomial parameter number
def polyParamsNumber(n, tau):
    return int(math.comb(n + tau, tau))


In [5]:

def loss(Y_true, Y_pred, THETA, lambda_param):
    """ Mean Squared Error """
    E = Y_true - Y_pred
    SSE = np.square(np.linalg.norm(E, 'fro'))
    Reg = (lambda_param/(2*E.shape[0]))*np.square(np.linalg.norm(THETA[1:,:], 'fro'))
    MSE = SSE/(2*E.shape[0]) + Reg
    return MSE


In [6]:
def gradient(A,E,THETA,lambda_param):
    """ MSE Gradient """
    SSEGrad = -1.0*(A.T).dot(E)
    MSEGrad = SSEGrad/E.shape[0]
    MSEGrad[1:,:] = MSEGrad[1:,:] + (lambda_param/E.shape[0])*THETA[1:,:]
    return MSEGrad



In [7]:
# AdamD Optimizer
def adamd_optimization(
    X,
    Y,
    tau,
    lambda_param=0.0,
    maxEpochs=100,
    show=10,
    batch_size=16,
    learning_rate=0.001,
    beta1=0.9,
    beta2=0.999,
    epsilon=1e-8,
    stopping_threshold=1e-6,
):
    n = X.shape[1]
    m = Y.shape[1]
    rho = polyParamsNumber(n, tau)
    THETA = np.random.randn(rho, m) * 0.01
    q = X.shape[0]
    
    # inicializar momentos
    m_t = np.zeros_like(THETA)
    v_t = np.zeros_like(THETA)
    
    t = 0
    previous_loss = np.inf

    for epoch in range(maxEpochs + 1):
        THETA_prev = THETA.copy()
        current_loss = np.inf

        if batch_size < q:
            indices = np.random.permutation(q)
            X = X[indices, :]
            Y = Y[indices, :]

        n_batches = q // batch_size
        residual = q % batch_size
        total_batches = n_batches + 1 if residual != 0 else n_batches

        for batch_idx in range(total_batches):
            t += 1  # incrementar timestep
            
            start = batch_idx * batch_size
            end = start + batch_size
            if batch_idx == total_batches - 1 and residual != 0:
                end = start + residual

            X_batch = X[start:end, :]
            Y_batch = Y[start:end, :]
            A_batch = designMatrix(tau, X_batch)
            Y_pred = model(A_batch, THETA)
            E_batch = Y_batch - Y_pred
            g_t = gradient(A_batch, E_batch, THETA, lambda_param)
            
            # actualizar momentos
            m_t = beta1 * m_t + (1 - beta1) * g_t
            v_t = beta2 * v_t + (1 - beta2) * (g_t ** 2)
            
            # calcular learning rate para este timestep
            alpha_t = learning_rate * np.sqrt(1 - beta2 ** t)
            
            # actualizar parametros
            THETA -= alpha_t * m_t / (np.sqrt(v_t) + epsilon)

        A = designMatrix(tau, X)
        Yh = model(A, THETA)
        current_loss = loss(Y, Yh, THETA, lambda_param)

        if epoch % show == 0:
            print(f"Epoch {epoch}: Loss={current_loss:.3e}, lr={alpha_t:.2e}")

        if abs(previous_loss - current_loss) < stopping_threshold:
            break
            
        previous_loss = current_loss

    return THETA

In [8]:
# Load dataset
mat = sp.loadmat('engine_dataset.mat')
inputs  = mat['engineInputs'].T
targets = mat['engineTargets'].T



In [9]:
# Train and Test Split Data
inputs_train, inputs_test, targets_train, targets_test = train_test_split(inputs, targets, random_state = 1, test_size = 0.4)


In [10]:
# Train and Test Data
xTrain = inputs_train
tTrain = targets_train
xTest = inputs_test
tTest = targets_test

In [11]:

#mini lote
# Find the optimal parameters with AdamD
tau = 2
lambda_param = 0.1

THETA = adamd_optimization(
    xTrain,
    tTrain,
    tau=tau,
    lambda_param=lambda_param,
    maxEpochs=10000,
    show=500,
    batch_size=64,
    learning_rate=0.001,
    beta1=0.9,
    beta2=0.999,
    epsilon=1e-8,
    stopping_threshold=1e-6,
)

Epoch 0: Loss=9.672e+07, lr=1.09e-04
Epoch 500: Loss=7.138e+04, lr=9.99e-04
Epoch 1000: Loss=6.377e+04, lr=1.00e-03
Epoch 1500: Loss=3.531e+04, lr=1.00e-03
Epoch 2000: Loss=7.423e+04, lr=1.00e-03
Epoch 2500: Loss=6.991e+04, lr=1.00e-03
Epoch 3000: Loss=6.734e+04, lr=1.00e-03
Epoch 3500: Loss=2.907e+04, lr=1.00e-03
Epoch 4000: Loss=3.246e+04, lr=1.00e-03
Epoch 4500: Loss=2.477e+04, lr=1.00e-03
Epoch 5000: Loss=2.254e+04, lr=1.00e-03
Epoch 5500: Loss=2.766e+04, lr=1.00e-03
Epoch 6000: Loss=4.406e+04, lr=1.00e-03
Epoch 6500: Loss=6.157e+04, lr=1.00e-03
Epoch 7000: Loss=1.653e+04, lr=1.00e-03
Epoch 7500: Loss=3.464e+04, lr=1.00e-03
Epoch 8000: Loss=3.564e+04, lr=1.00e-03
Epoch 8500: Loss=1.840e+04, lr=1.00e-03
Epoch 9000: Loss=1.502e+04, lr=1.00e-03
Epoch 9500: Loss=4.339e+04, lr=1.00e-03
Epoch 10000: Loss=1.613e+04, lr=1.00e-03


In [12]:

# Make predictions
# Train data
A_train = designMatrix(tau,xTrain)
outputTrain = model(A_train,THETA)
# Test data
A_test = designMatrix(tau,xTest)
outputTest = model(A_test,THETA)

In [13]:

# Metrics
# Train data
R2_train = r2_score(tTrain.reshape(-1, 1), outputTrain.reshape(-1, 1))
print(R2_train)

0.9381630934043915


In [14]:
MSE_train = mean_squared_error(tTrain.reshape(-1, 1), outputTrain.reshape(-1, 1))
print(MSE_train)


16126.81710705816


In [15]:

# Test data
R2_test = r2_score(tTest.reshape(-1, 1), outputTest.reshape(-1, 1))
print(R2_test)

0.9424683362277272


In [16]:
MSE_test = mean_squared_error(tTest.reshape(-1, 1), outputTest.reshape(-1, 1))
print(MSE_test)


16261.71687615213


In [17]:
THETA

array([[-9.64145697e-01,  6.79223419e-01],
       [ 4.93389517e+00,  1.25978079e+01],
       [-1.73465728e-03, -3.03313080e-02],
       [-3.93973210e-02,  3.71400335e-01],
       [ 1.22338865e-03,  1.25363043e-04],
       [-4.99357791e-05, -2.97922822e-04]])